In [1]:
!pip install -q orb-models
!pip install -q ase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 853.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13

In [6]:
!pip install gemmi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.6 MB/s eta 0:00:00


In [13]:
!pip install pynanoflann

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 3.9 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/aglie/seminar_disorder_2025.git

Cloning into 'seminar_disorder_2025'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 47 (delta 17), reused 19 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 2.50 MiB | 7.03 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [4]:
cd seminar_disorder_2025/

/content/seminar_disorder_2025


In [9]:
from CalculateScattering import CrystalStructure
import numpy as np

In [10]:
# ----- Convert Lattice to Crystal Structure -----
def convert_lattice_to_crystal_structure(lattice, a=3.0):
    """
    Convert a 2D lattice of +1 (Cu) and -1 (Au) into a 3D CrystalStructure
    """
    height, width = lattice.shape

    # Create atoms list (element_type, x, y, z)
    atoms = []
    for x in range(height):
        for y in range(width):
            # Convert +1 to Cu, -1 to Au
            element = "Cu" if lattice[x, y] == 1 else "Au"
            atoms.append((element, x, y, 0))

    # Create crystal structure with a simple cubic unit cell
    crystal = CrystalStructure(
        cell_parameters=(a, a, a, 90, 90, 90),  # Simple cubic cell
        atoms=atoms,
        supercell=(width, height, 1)  # 2D structure with thickness of 1
    )

    return crystal

In [14]:
from orb_models.forcefield import pretrained
from orb_models.forcefield.calculator import ORBCalculator

In [15]:
device = "cpu"  #or cuda or tpu
orbff = pretrained.orb_v2(device=device)
calc = ORBCalculator(orbff, device=device)

Output()

In [19]:
cu_lattice = np.array([[1, 1], [1, 1]])
crystal = convert_lattice_to_crystal_structure(cu_lattice)

crystal_atoms_format = crystal.to_ase_atoms()
crystal_atoms_format.calc = calc

energy = crystal_atoms_format.get_potential_energy()
print(f"The energy of the crystal structure is: {energy} eV")

The energy of the crystal structure is: -10.578989028930664 eV


# Exercise: calculate mixing

Now using this pattern calculate the energy of mixing Cu with Au:

- Create a larger supercell with only 2 Au atoms which are separated by Cu
- And another one where 2 Au atoms are close to each other
- Calculate the energy of both configurations
- Assume the energy of interaction is only nearest neighbour. What should be the value of J to account for such a pair?